In [1]:
# Import Python package
import binhopulsar
from binhopulsar.pulsar import Pulsar
from binhopulsar.commands.spi.definitions import *

## Getting started

#### 1. List all the Pulsar devices connected to the PC host

The ``binhopulsar.getConnectedPulsarDevicesList()`` gets a list of the Pulsar devices plugged into the host PC machine.

In [2]:
binhopulsar.getConnectedPulsarDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FD#7&271efd59&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fd',
  'serial_number': '0AA4AA0BBF431055B02059D06095F7BB',
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Pulsar',
  'hardware_version': 'B',
  'firmware_version': '4.1.2'}]

#### 2. Create an instance of the Pulsar class

To utilize a Pulsar USB host adapter device, we need to create an instance of the Pulsar class.

In [6]:
# Create a device instance. One instance of the Pulsar class represents a Pulsar USB host adapter device.
pulsar = Pulsar()

#### 3. Open connection to the Pulsar device

The public method ``Pulsar.open()`` establishes the connection with a Pulsar device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Pulsar serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhopulsar.getConnectedPulsarDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Pulsar device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Pulsar device simultaneously.


In [7]:
# Use the method by default to connect to only one Pulsar device.
pulsar.open()

# Otherwise, use the path attribute to identify each Pulsar device. Uncomment the line below and comment the line #2.
# pulsar.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Pulsar device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Pulsar

To handle responses and notifications from Pulsar, a callback function must be defined and registered. This function will be invoked every time the Pulsar sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(pulsar_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Pulsar.onEvent(callback_function)`` method.

In [8]:
# Define callback function
def callback_function(pulsar_message: dict, system_message: dict) -> None:

    if pulsar_message != None:

        # Print message
        print(">> New message from PULSAR:")
        print(pulsar_message)

    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [9]:
# Register callback function
pulsar.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Pulsar from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [10]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## SPI Protocol API

The SPI Protocol API methods are described below.

### 1. In the first case, we will run the examples in a loopback by shorting MOSI and MISO.

``Pulsar.setI2cSpiUartGpioVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV. It also allows setting the value 0 mV to power off the output voltage.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300] mV and allows the value 0 mV to power off the output voltage.

In [11]:
pulsar.setI2cSpiUartGpioVoltage(getId(), 3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART GPIO VOLTAGE request success'}

>> New message from PULSAR:
{'id': 1, 'command': 'SYS SET I2C SPI UART GPIO VOLTAGE', 'result': 'SUCCESS'}


``Pulsar.spiControllerInit(id, bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)``

This method initialize the SPI peripheral and must be provided with configuration parameters as shown below.

  - ``id:`` A 2-byte integer that represents the transfer ID.
  - ``bitOrder:`` Sets the bit order of the SPI transfers.
  - ``mode:`` Sets the SPI mode.
  - ``dataWidth:`` Sets the SPI dataWidth. Currently, only supports 8 bits data width transfers
  - ``chipSelect:`` Allows to select different chips to communicate with.
  - ``chipSelectPol:`` Sets the polarity used to enable the selected chip.
  - ``frequency:`` Represents the SPI SCK frequency expressed in Hz. Currently, the minimum allowed value is 10000 Hz and the maximum allowed value is 50000000 Hz.

In [12]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                        # 10 MHz

pulsar.spiControllerInit(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1, 'opcode': 0, 'message': 'SPI CONTROLLER INIT request success'}

>> New message from PULSAR:
{'id': 2, 'command': 'SPI CONTROLLER INIT', 'result': 'SUCCESS'}


``Pulsar.spiControllerTransfer(id, transferLength, payload: list)``

This method is used to request to the the Pulsar device to perform a SPI transfer.

  - ``id: int`` : It is a 2-bytes integer that represents the transfer id.
  - ``payload: list`` : A Python list that contains the SPI data to be transferred. The list holds bytes elements (0x00 - 0xFF), and the maximum length is 1024 bytes.
  - ``transferLength: int`` : It is a 2-bytes integer that represents the transfer length including the data to transfer. In case of an opcode instruction of a read, the transfer length represents the length of the opcode and the size of bytes to be read

In [13]:
payload = [i%0xFF for i in range(0,1024)]
total_transfer_length = 1024

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:


{'id': 3, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,

``Pulsar.spiControllerSetParameters(id, bitOrder=SpiControllerBitOrder.LSB, mode=SpiControllerMode.MODE_2, chipSelect=SpiControllerChipSelect.CHIP_SELECT_1, chipSelectPol=SpiControllerChipSelectPolarity.ACTIVE_HIGH, frequency=30000000)``

This method is used to change the initialized configuration.

  - ``id``: A 2-byte integer that represents the transaction ID.
  - ``bitOrder``: Sets the bit order of the SPI transfer.
  - ``mode``: Sets the SPI mode.
  - ``dataWidth:`` Sets the SPI dataWidth.
  - ``chipSelect:`` Allows to select different chips to communicate with.
  - ``chipSelectPol``: Sets the polarity used to enable the selected chip.
  - ``frequency``: Represents the SPI SCK frequency expressed in Hz. Currently, the minimum allowed value is 10000 Hz and the maximum allowed value is 50000000 Hz, which corresponds to 50 MHz.

Note: it is necessary to send all the configuration parameter even if they keep the same value as they had before

Example changing ``bitOrder`` from ``MSB`` to ``LSB`` first

In [14]:
bitOrder = SpiControllerBitOrder.LSB                        # LSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                         # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

>> New message from PULSAR:
{'id': 4, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

In [15]:
payload = [i%0xFF for i in range(0,1024)]
total_transfer_length = 1024

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 5, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 19

Example of transfer in ``MODE_1``

CPOL = 0  |  CPHA = 1

In [16]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_1                             # Mode 1
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                        # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

>> New message from PULSAR:
{'id': 6, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

In [17]:
payload = [i%0xFF for i in range(0,1024)]
total_transfer_length = 1024

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 7, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 19

Example of transfer in ``MODE_2``

CPOL = 1  |  CPHA = 0

In [18]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_2                             # Mode 2
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                         # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 8, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [19]:
payload = [i%0xFF for i in range(0,1024)]
total_transfer_length = 1024

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:


{'id': 9, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,

Example of transfer in ``MODE_3``

CPOL = 1  |  CPHA = 1

In [20]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_3                             # Mode 3
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                         # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 10, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [21]:
payload = [i%0xFF for i in range(0,500)]
total_transfer_length = 500

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 11, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 500, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 19

Example of transfer with ``CHIP_SELECT_1``

In [22]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_1          # Chip select 1
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                         # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 12, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [23]:
payload = [i%0xFF for i in range(0,10)]
total_transfer_length = 10

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 13, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 10, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


Example of transfer with ``CHIP_SELECT_2``

In [24]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_2          # Chip select 2
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                         # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 14, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [25]:
payload = [i%0xFF for i in range(0,10)]
total_transfer_length = 10

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

>> New message from PULSAR:

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}


{'id': 15, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 10, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


Example of transfer with ``CHIP_SELECT_3``

In [26]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_3          # Chip select 3
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                        # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 16, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [27]:
payload = [i%0xFF for i in range(0,10)]
total_transfer_length = 10

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 17, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 10, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


Example of transfer with ``CHIP SELECT POLARITY: ACTIVE HIGH``

In [28]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 1
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_HIGH # Active high
frequency = 10000000                                         # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

>> New message from PULSAR:

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}


{'id': 18, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [29]:
payload = [i%0xFF for i in range(0,10)]
total_transfer_length = 10

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

>> New message from PULSAR:

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}


{'id': 19, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 10, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


Example of transfer with ``BAUDRATE: 10 kHz``

In [30]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000                                           # 10 kHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 20, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [31]:
payload = [i%0xFF for i in range(0,10)]
total_transfer_length = 10

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 21, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 10, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


Example of transfer with ``BAUDRATE: 50 MHz``

In [32]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 50000000                                         # 50 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 22, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In [33]:
payload = [i%0xFF for i in range(0,100)]
total_transfer_length = 100

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

>> New message from PULSAR:

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}


{'id': 23, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 100, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]}


### 2. Now, let's connect the FRAM memory MB85RS64V

In [34]:
bitOrder = SpiControllerBitOrder.MSB                        # MSB first
mode = SpiControllerMode.MODE_0                             # Mode 0
dataWidth = SpiControllerDataWidth._8_BITS_DATA             # 8 bits data width
chipSelect = SpiControllerChipSelect.CHIP_SELECT_0          # Chip select 0
chipSelectPol = SpiControllerChipSelectPolarity.ACTIVE_LOW  # Active low
frequency = 10000000                                        # 10 MHz

pulsar.spiControllerSetParameters(getId(), bitOrder, mode, dataWidth, chipSelect, chipSelectPol, frequency)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 24, 'command': 'SPI CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


In the next case, the transfer command It's used to send a Read Manufacturer ID instruction to a FRAM memory. The ``payload`` contains the opcode of the instruction in a list of 1 byte and the ``transferLength`` includes the opcode length and the command response that It's 4 bytes, so the ``transferLength`` parameter has to be 5. This command returns the values [0x04, 0x7F, 0x03, 0x02] or in decimal notation [4, 127, 3, 2]

In [35]:
payload = [0x9F]                # opcode for Read Manufacturer ID
total_transfer_length = 5       # 5 bytes including the command and the response

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 25, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 5, 'payload': [255, 4, 127, 3, 2]}


The next instruction enables the write operation

In [36]:
payload = [0x06]                # opcode for WREN command
total_transfer_length = 1       # 1 byte just for the command

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 26, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 1, 'payload': [255]}


The next instruction writes data to FRAM memory cell array. The structure is: WRITE op-code (0x02), arbitrary 16 bits of address and 8 bits writing data

In [37]:
payload = [0x02, 0x00, 0x00, 0x2A, 0x3B]   # opcode for WRITE command (0x02), address (0x0000), data (0x2A3B)
total_transfer_length = 5                  # 5 bytes including the command, address and data

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 27, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 5, 'payload': [255, 255, 255, 255, 255]}


The next instruction reads FRAM memory cell array. The structure is: READ op-code (0x03) and arbitrary 16 bits of address. In this case, it will read two bytes from the address 0x0000 and the expected values are [0x2A, 0x3B] corresponding with the previous write command (In decimal notation [42, 59])

In [38]:
payload = [0x03, 0x00, 0x00]   # opcode for READ command (0x03), address (0x0000)
total_transfer_length = 5      # 5 bytes including the command, address and data size to be read

pulsar.spiControllerTransfer(getId(), total_transfer_length, payload)

{'module': 1,
 'opcode': 0,
 'message': 'SPI CONTROLLER TRANSFER request success'}

>> New message from PULSAR:
{'id': 28, 'command': 'SPI CONTROLLER TRANSFER', 'result': 'SUCCESS', 'payload_length': 5, 'payload': [255, 255, 255, 42, 59]}


## Close communication

Use the ``Pulsar.close()`` method to end the communication with the Pulsar device and release the used memory in the background like threads and so on.

In [39]:
# Close the communication with the Pulsar device.
pulsar.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}